In [ ]:
!apt install libcublas11

In [ ]:
!pip install faster-whisper

In [ ]:
from faster_whisper import WhisperModel
import os
from google.colab import files

# 为什么用v3,有新的当然用新的喽
model_size = "large-v3" # tiny, base, small, medium, large, large-v2, large-v3
mode = "normal" # normal 一般, timeline 加入时间轴, subtitle 产生字幕格式

# Run on GPU with FP16
model = WhisperModel(model_size, device="cuda", compute_type="float16")

# 路径
audio_path = "/content/1.wav"

segments, info = model.transcribe(audio_path, beam_size=5, initial_prompt="简体")
# 繁體
# segments, info = model.transcribe(audio_path, beam_size=5, initial_prompt="繁體")

transcription = ""

# 1 以下一般版本
if mode == "normal":
  transcription_segments = [segment.text for segment in segments]
  transcription = "，".join(transcription_segments)

# 2 以下加入时间轴版本
elif mode == "timeline":
  for segment in segments:
    transcription += "[%.2fs -> %.2fs] %s\n" % (segment.start, segment.end, segment.text)

# 3 以下产生字幕格式的版本
elif mode == "subtitle":
  for i, segment in enumerate(segments, 1):
    start_hours, start_remainder = divmod(segment.start, 3600)
    start_minutes, start_seconds = divmod(start_remainder, 60)
    end_hours, end_remainder = divmod(segment.end, 3600)
    end_minutes, end_seconds = divmod(end_remainder, 60)
    transcription += "%d\n%02d:%02d:%06.3f --> %02d:%02d:%06.3f\n%s\n\n" % (
      i,
      start_hours, start_minutes, start_seconds,
      end_hours, end_minutes, end_seconds,
      segment.text
    )

print(transcription)

file_name = os.path.splitext(os.path.basename(audio_path))[0]

# 结果保存txt
with open(f"{file_name}.txt", "w") as file:
  file.write(transcription)
  files.download(f"{file_name}.txt")